In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [11]:
# url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
tt = pd.read_csv("titanic.csv")
print(tt.columns)

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
target = "Survived"

Index(['Unnamed: 0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age',
       'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [12]:
# 处理缺失值
tt["Age"].fillna(tt["Age"].mean(), inplace=True)
tt["Embarked"].fillna(tt["Embarked"].mode()[0], inplace=True)
tt["Sex"] = tt["Sex"].map({'male':1, 'female':0})

x = tt[features]
y = tt[target]

In [13]:
x_trn, x_val, y_trn, y_val = train_test_split(x, y, test_size=0.2, random_state=21)

clf = DecisionTreeClassifier(random_state=21)
clf.fit(x_trn, x_val)

/home/nekozo/newSpace/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/nekozo/newSpace/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/nekozo/newSpace/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


ValueError: could not convert string to float: 'C'

In [ ]:
y_pred = clf.predict(x_val)
acc = accuracy_score(y_val, y_pred)
print(f"accuracy: {acc}")